#Setup
Specify your desired blender version and the path to your blend file within google drive or colab local storage.

###Info
If you do need more information on parameters etc. look here: [Blender CLI Wiki](https://docs.blender.org/manual/en/latest/advanced/command_line/arguments.html)

In [20]:
#@title Setup
#@markdown Please configure your setup

blender_version = 'blender2.91.2' #@param ["blender2.80", "blender2.81", "blender2.82", "blender2.83", "blender2.90.1", "blender2.91.2"] {allow-input: false}
#@markdown You can right click a file and click copy path
path_to_blend = '/content/blendfile.blend'  #@param {type: "string"}
output_path = '/content/output'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
use_drive = False #@param {type:"boolean"}
#@markdown ---
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  

In [4]:
if blender_version == "blender2.80":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.80/blender-2.80-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.81":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82-linux64.tar.xz"
elif blender_version == "blender2.83":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.83/blender-2.83.0-linux64.tar.xz"
elif blender_version == "blender2.90.1":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz"
elif blender_version == "blender2.91.2":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz"

In [ ]:
!mkdir $blender_version
if blender_version == "blender2.81" or "blender2.80":
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1


This Block is required as some weird behaviors with libtcmalloc appeared in the colab VM

In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

Unpack and move blender

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

Required for Blender to use the GPU as expected

In [8]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

Use this if you want to render all Frames

Use -s to speficy the start frame.
eg: -s 10

Use -e to speficy the end frame.
eg: -e 20

**THE ORDER IS IMPORTANT. BOTH -s AND -e MUST BE SPEFICIED BEFORE -a**

In [18]:
#@title Render Setup
#@markdow
whatToRender = 'renderOneFrame' #@param ["renderOneFrame", "renderAllFrame", "renderWithinRange"] {allow-input: false}
#@markdown StartFrame = the frame you want to start with or the frame number for the renderOneFrame option
startFrame =  1#@param {type: "integer"}
endFrame = 20 #@param {type: "integer"}

In [ ]:
if whatToRender == 'renderOneFrame':
  !sudo ./$blender_version/blender -P 'setgpu.py' -b '{path_to_blend}' -P 'setgpu.py' -o '{output_path}' -f {startFrame}
elif whatToRender == 'renderAllFrame':
  !sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -a
elif whatToRender == 'renderWithinRange':
  !sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -s {startFrame} -e {endFrame} -a